# FuseTS - Phenology example


## Setting up the OpenEO process
The first step includes setting up the OpenEO processing through the [OpenEO Python Client](https://open-eo.github.io/openeo-python-client/). Since the Whittaker algorithm is integrated as an [user defined process](https://open-eo.github.io/openeo-python-client/cookbook/udp_sharing.html), we can use the `datacube_from_process` function to execute the function.

In [1]:
import openeo
import json
import pandas as pd
import xarray
import matplotlib.pyplot as plt
from openeo.rest.conversions import timeseries_json_to_pandas
from openeo.metadata import CollectionMetadata
from shapely.geometry import box

In [2]:
connection = openeo.connect("openeo.vito.be").authenticate_oidc()
service = 'phenology'
namespace = 'u:bramjanssen'

Authenticated using refresh token.


In [3]:
connection.describe_process(service, namespace=namespace)

{'description': '# Phenology\n\n## Description\n\n\n',
 'id': 'phenology',
 'parameters': [{'description': 'A data cube.',
   'name': 'data',
   'schema': {'subtype': 'raster-cube', 'type': 'object'}}],
 'summary': ''}

In [4]:
spat_ext = {
          "type": "Polygon",
          "coordinates": [
            [
              [
                5.170012098271149,
                51.25062964728295
              ],
              [
                5.17085904378298,
                51.24882567194015
              ],
              [
                5.17857421368097,
                51.2468515482926
              ],
              [
                5.178972704726344,
                51.24982704376254
              ],
              [
                5.170012098271149,
                51.25062964728295
              ]
            ]
          ]
        }
temp_ext = ["2022-01-01","2022-12-31"]
smoothing_lambda = 10000

To begin, we calculate the base NDVI data cube. In the subsequent steps, we utilize this data cube to implement Whittaker smoothing. Finally, we compute time series for both the base NDVI and the smoothed version.

In [5]:
base = connection.load_collection('SENTINEL2_L2A_SENTINELHUB',
                                spatial_extent=spat_ext,
                                temporal_extent=temp_ext,
                                bands=["B04","B08","SCL"])
base_cloudmasked = base.process("mask_scl_dilation", data=base, scl_band_name="SCL")
base_ndvi = base_cloudmasked.ndvi(red="B04", nir="B08")

In [6]:
phenology = connection.datacube_from_process(service, namespace=f'https://openeo.vito.be/openeo/1.1/processes/{namespace}/{service}', data=base_ndvi)

/home/bramjanssen/.local/lib/python3.6/site-packages/openeo/metadata.py:255: UserWarning: No cube:dimensions metadata
  complain("No cube:dimensions metadata")


In [11]:
phenology_job = phenology.execute_batch(out_format="netcdf", title=f'FuseTS - Phenology', job_options={
    'udf-dependency-archives': [
         'https://artifactory.vgt.vito.be:443/auxdata-public/ai4food/fusets_venv.zip#tmp/venv',
        'https://artifactory.vgt.vito.be:443/auxdata-public/ai4food/fusets.zip#tmp/venv_static'
    ]
})
phenology_job.get_results().download_file('./phenology.nc')

0:00:00 Job 'j-69124a1f99934e9f9888989daad62010': send 'start'
0:00:22 Job 'j-69124a1f99934e9f9888989daad62010': queued (progress N/A)
0:00:27 Job 'j-69124a1f99934e9f9888989daad62010': queued (progress N/A)
0:00:33 Job 'j-69124a1f99934e9f9888989daad62010': queued (progress N/A)
0:00:41 Job 'j-69124a1f99934e9f9888989daad62010': queued (progress N/A)
0:00:51 Job 'j-69124a1f99934e9f9888989daad62010': running (progress N/A)
0:01:04 Job 'j-69124a1f99934e9f9888989daad62010': running (progress N/A)
0:01:23 Job 'j-69124a1f99934e9f9888989daad62010': running (progress N/A)
0:01:42 Job 'j-69124a1f99934e9f9888989daad62010': running (progress N/A)
0:02:06 Job 'j-69124a1f99934e9f9888989daad62010': running (progress N/A)
0:02:36 Job 'j-69124a1f99934e9f9888989daad62010': running (progress N/A)
0:03:14 Job 'j-69124a1f99934e9f9888989daad62010': error (progress N/A)
Your batch job 'j-69124a1f99934e9f9888989daad62010' failed. Error logs:
[{'id': '[1685633154434, 132778]', 'time': '2023-06-01T15:25:54.434Z

JobFailedException: Batch job 'j-69124a1f99934e9f9888989daad62010' didn't finish successfully. Status: error (after 0:03:14).

## Visualising the results

In [10]:
phenology_result = xarray.load_dataset('./phenology.nc')
phenology_result.var

<bound method ImplementsDatasetReduce._reduce_method.<locals>.wrapped_func of <xarray.Dataset>
Dimensions:  (t: 1, x: 65, y: 45)
Coordinates:
  * t        (t) datetime64[ns] 2020-01-01
  * x        (x) float64 6.514e+05 6.514e+05 6.515e+05 ... 6.521e+05 6.521e+05
  * y        (y) float64 5.68e+06 5.68e+06 5.68e+06 ... 5.68e+06 5.68e+06
Data variables:
    crs      |S1 b''
    var      (t, y, x) float64 0.8873 0.8689 0.8909 ... 0.8908 0.8858 0.9385
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.11.0a1
    description:  
    title:        >

In [ ]:
sos = phenology_result.sos_times.median(dim=['x', 'y']).values.take(0)
pos = phenology_result.pos_times.median(dim=['x', 'y']).values.take(0)
eos = phenology_result.eos_times.median(dim=['x', 'y']).values.take(0)

sos_date = datetime.datetime(year, 1, 1) + datetime.timedelta(days=sos)
pos_date = datetime.datetime(year, 1, 1) + datetime.timedelta(days=pos)
eos_date = datetime.datetime(year, 1, 1) + datetime.timedelta(days=eos)
plt.legend()